In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [2]:
peaks_data = pd.read_csv("peaks_tss_distance.csv")

In [3]:
peaks_data.head()

,ImmGenATAC1219.peakID,chrom,peak_center,closest_tss,distance_to_tss
0,ImmGenATAC1219.peak_1,chr1,3020786,3671498.0,650712.0
1,ImmGenATAC1219.peak_2,chr1,3087226,3671498.0,584272.0
2,ImmGenATAC1219.peak_3,chr1,3120109,3671498.0,551389.0
3,ImmGenATAC1219.peak_4,chr1,3121485,3671498.0,550013.0
4,ImmGenATAC1219.peak_5,chr1,3372787,3671498.0,298711.0


In [4]:
peaks_data["RegionType"] = peaks_data["distance_to_tss"].apply(
    lambda x: "Promoter" if x <= 1000 else "Enhancer"
)

In [5]:
peaks_data.head()

,ImmGenATAC1219.peakID,chrom,peak_center,closest_tss,distance_to_tss,RegionType
0,ImmGenATAC1219.peak_1,chr1,3020786,3671498.0,650712.0,Enhancer
1,ImmGenATAC1219.peak_2,chr1,3087226,3671498.0,584272.0,Enhancer
2,ImmGenATAC1219.peak_3,chr1,3120109,3671498.0,551389.0,Enhancer
3,ImmGenATAC1219.peak_4,chr1,3121485,3671498.0,550013.0,Enhancer
4,ImmGenATAC1219.peak_5,chr1,3372787,3671498.0,298711.0,Enhancer


In [8]:
print(peaks_data["distance_to_tss"].min())
print(peaks_data["distance_to_tss"].max())

0.0
2012626.0


In [ ]:
sns.violinplot